In [1]:
import json
from pprint import pprint
from document_analitics_functions import *
%matplotlib inline
import pandas as pd


data_dir = './data/skill-bancolombia_tabot_cert.json'
labels = []
texts = []
data_dict={}
min_utt=0
intent_count=0
stop_word_list=['a','e','ya','pas','ni','nah','qu','al','ese','lee','se','con','y','lo','ha','he','son','mis','soy','yo','o','si','no','de','del','en','que','qué','mi','un','por','es','una','me','te','la','los','el','las','como','cómo','este','estan','quisiera','favor','cual','cuál','cuales','esta','para','ayuda','ayudas','necesito','favor','puedo','quiero', 'hago','donde', 'tener', 'pedir','quisiera','cuales','tiene','tengo','tener','hacer','debo','quiero','puedes','indicar','realizar','solicitud','solicitar','estos','saber','hasta','hace','desde','consulta','consultar','puede','pregunta','preguntas','hace','info','sobre','porque','cada','realiza','realizar','otro','otros','tabot','pero','esto']    
scatter_data = np.empty((0, 4)) # four dimension scatter data
    
with open(data_dir, encoding='utf-8') as data_file:
    data = json.loads(data_file.read())  
    
#pprint(data['intents'][0]['intent'])
#pprint(data['intents'][0]['examples'][0]['text'])    

for intent in data['intents']:
    #print(intent['intent'])
    utt_count=0
    if len(intent['examples'])>min_utt: #intents con mínimo min_utt utterances          
        texts_intent=[]    
        for utt_count, utterance in enumerate(intent['examples']):
            labels.append(str(intent['intent']))
            texts.append(utterance['text'])
            texts_intent.append(utterance['text'])
            utt_count += 1
#             if utt_count > min_utt:
#                 break
        data_dict[intent['intent']]=texts_intent  # list of values for each key 
        intent_count += 1
        #print(utt_count, intent_count)
#print(intent_list) 


In [ ]:
import json
from pprint import pprint
from document_analitics_functions import *
%matplotlib inline
import pandas as pd


data_dir = './data/tabot_logs1.csv'
data_dict={}
stop_word_list=['a','e','ya','pas','ni','nah','qu','al','ese','lee','se','con','y','lo','ha','he','son','mis','soy','yo','o','si','no','de','del','en','que','qué','mi','un','por','es','una','me','te','la','los','el','las','como','cómo','este','estan','quisiera','favor','cual','cuál','cuales','esta','para','ayuda','ayudas','necesito','favor','puedo','quiero', 'hago','donde', 'tener', 'pedir','quisiera','cuales','tiene','tengo','tener','hacer','debo','quiero','puedes','indicar','realizar','solicitud','solicitar','estos','saber','hasta','hace','desde','consulta','consultar','puede','pregunta','preguntas','hace','info','sobre','porque','cada','realiza','realizar','otro','otros','tabot','pero','esto']    
scatter_data = np.empty((0, 4)) # four dimension scatter data

df = pd.read_csv(data_dir,usecols=[1,2])
labels=df['intent'].tolist()
texts =df['phrase'].tolist()
for i in df['intent'].unique():
    data_dict[i] = [df['phrase'][j] for j in df[df['intent']==i].index]
#print(data_dict)

In [3]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = 6000
n_features = 1000
n_components = 3
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

# print("Loading dataset...")
# t0 = time()
# dataset = fetch_20newsgroups(shuffle=True, random_state=1,
#                              remove=('headers', 'footers', 'quotes'))
# data_samples = dataset.data[:n_samples]
# print("done in %0.3fs." % (time() - t0))

data_samples=texts

# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words=stop_word_list)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words=stop_word_list)
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf-idf features for NMF...
done in 0.077s.
Extracting tf features for LDA...
done in 0.047s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=6000 and n_features=1000...
done in 0.188s.

Topics in NMF model (Frobenius norm):
Topic #0: tarjeta credito crédito pagar bloquear cancelar cuanto debito eprepago cuota mes prepago activar fecha corte bloqueo valor perdi saldo cuando
Topic #1: cuenta clave cambiar movimientos numerode numero certificado ahorros transferencia bancolombia cambio conocer podrias celular bancaria hola inscrita decir ahorro otra
Topic #2: pago total deuda fecha valor mínimo cuanto minimo pagar tdc cobrando extracto corte mes entiendo monto conocer movimientos cuota intereses

Fitting the NMF model (generalized Kullback-Leibler divergence) with tf-idf features, n_samples=6000 and n_features=1000...
done in 0.350s.

Topics in NMF model (generalized Kullback-Leibler divergence):
Topic #0: tarjeta credito crédito tarjetas eprepago hay ca